In [2]:
import pandas as pd
import nltk
import string
import numpy as np

In [3]:
articles = []

for l in range(1,4):
    k = 32
    if (l == 2):
        k=30     
    for i in range(1,k):
        dataP = pd.read_csv(str(l) + "_" + str(i) + "_P.csv")
        dataTH = pd.read_csv(str(l) + "_" + str(i) + "_TH.csv")
        dataTOI = pd.read_csv(str(l) + "_" + str(i) + "_TOI.csv")
        for j in range(len(dataP)):
            articles.append(dataP['Article'][j])
        for j in range(len(dataTH)):
            articles.append(dataTH['Content'][j])
        for j in range(len(dataTOI)):
            articles.append(dataTOI['Content'][j])
                     
     

In [4]:
print(len(articles))

105006


In [5]:
def to_lowercase(articles):
    temp = []
    for article in articles:
        if type(article) is str:
            temp.append(article.lower())
    return temp
articles = to_lowercase(articles)
    

In [6]:
for i in range(len(articles)):
    articles[i] = articles[i].translate(str.maketrans('','' , string.punctuation))

In [7]:
for i in range(len(articles)):
    articles[i] = " ".join(articles[i].split())

In [8]:
from nltk import word_tokenize
words = []


In [9]:
for i in range(len(articles)):
    words.append(word_tokenize(articles[i]))

In [10]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

In [11]:
for i in range(len(words)):
    words[i] = [word for word in words[i] if word not in stop_words]  

In [12]:
for i in range(len(words)):
    words[i] = [token for token in words[i] if not token.isnumeric()]

In [13]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
for i in range(len(words)):
    words[i] = [lemmatizer.lemmatize(j) for j in words[i]]

In [14]:
for i in range(len(words)): 
    words[i] = [token for token in words[i] if len(token) > 1]

In [ ]:
words[0]

In [15]:
from gensim.models import Phrases

In [16]:
bigram = Phrases(words, min_count=5,threshold= 100)

In [17]:
for idx in range(len(words)):
    for token in bigram[words[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            words[idx].append(token)    



In [18]:
from gensim.corpora import Dictionary

In [19]:
dictionary = Dictionary(words)
dictionary.filter_extremes(no_below =500 ,no_above=0.3)


In [26]:
print(len(dictionary))

4201


In [20]:
corpus = [dictionary.doc2bow(doc) for doc in words]


In [21]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [29]:
from gensim.models import LdaModel
num_topics = 6
chunksize = 5000
passes = 20
iterations = 400
eval_every = None

In [30]:
id2word = dictionary.id2token
print(len(id2word))

0


In [32]:
model = LdaModel(
    corpus=corpus_tfidf,
    id2word=dictionary,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [33]:
import pyLDAvis.gensim

In [34]:
lda_display = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary, sort_topics=False)


C:\Users\ajain\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [35]:
pyLDAvis.display(lda_display)


In [1]:
#print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=words, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

NameError: name 'CoherenceModel' is not defined

In [40]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [40]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_tfidf, num_topics=6, id2word=dictionary)

In [46]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus_tfidf, texts=words, start=2, limit=20, step=6)


In [36]:
from scipy.spatial import distance

In [ ]:
distance.jensenshannon(doc1,doc2)

In [23]:
from gensim.test.utils import datapath
from gensim.models import LdaModel
    
temp_file = datapath("model2")
#model.save((temp_file))

model2 = LdaModel.load(temp_file)

In [42]:
print(temp_file)
#model_mar = LdaModel.load(temp_file)

C:\Users\ajain\Anaconda3\lib\site-packages\gensim\test\test_data\modell


In [ ]:
from gensim.models import hdpmodel
hdp_model = hdpmodel.HdpModel(corpus=corpus_tfidf, id2word=dictionary)
hdp_model.print_topics(num_topics=-1)

In [ ]:
shown_topics = hdp_model.show_topics(num_topics=hdp_model.m_T, formatted=False)
topics_nos = [x[0] for x in shown_topics ]
weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]
ll= pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

In [ ]:
for i in range(0,150):
    print(ll['weight'][i])
    print('\n')

In [45]:
train_vecs = []
for i in range(len(words)):
    top_topics = model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(6)]
    train_vecs.append(topic_vec)

In [46]:
print(train_vecs[0])

[6.304063e-05, 5.930225e-05, 0.19166397, 4.6081892e-05, 0.80810606, 6.153711e-05]


In [24]:
test_articles = []

dataP = pd.read_csv("4_1_P.csv")
dataTH = pd.read_csv("4_1_TH.csv")
dataTOI = pd.read_csv("4_1_TOI.csv")
for j in range(len(dataP)):
    test_articles.append(dataP['Article'][j])
for j in range(len(dataTH)):
    test_articles.append(dataTH['Content'][j])
for j in range(len(dataTOI)):
    test_articles.append(dataTOI['Content'][j])
                     
 

In [25]:
print(len(test_articles))

1158


In [27]:
for i in range(len(test_articles)):
    if type(test_articles[i]) is not str:
        del test_articles[i]
    test_articles[i] = test_articles[i].lower()

In [28]:
for i in range(len(test_articles)):
    test_articles[i] = test_articles[i].translate(str.maketrans('','' , string.punctuation))

In [29]:
for i in range(len(test_articles)):
    test_articles[i] = " ".join(test_articles[i].split())

In [30]:
test = []
for i in range(len(test_articles)):
    test.append(word_tokenize(test_articles[i]))

In [31]:
stop_words = set(stopwords.words("english"))
for i in range(len(test)):
    test[i] = [word for word in test[i] if word not in stop_words]
for i in range(len(test)):
    test[i] = [token for token in test[i] if not token.isnumeric()]
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
for i in range(len(test)):
    test[i] = [lemmatizer.lemmatize(j) for j in test[i]]
for i in range(len(test)):    
    test[i] = [token for token in test[i] if len(token) > 1]

In [32]:
testcorpus = [dictionary.doc2bow(doc) for doc in test]


In [34]:
test_vecs = []
for i in range(len(test)):
    top_topics = model2.get_document_topics(testcorpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(6)]
    test_vecs.append(topic_vec)

In [35]:
topic0 = []
topic1 = []
topic2 = []
topic3 = []
topic4 = []
topic5 = []
matched_topic = [topic0, topic1, topic2, topic3, topic4, topic5]


In [36]:
for i in range(len(test_vecs)):
    mx = -1
    for j in range(len(test_vecs[i])):
        mx = max(mx, test_vecs[i][j])
    
    for j in range(len(test_vecs[i])):
        if mx==test_vecs[i][j]:
            matched_topic[j].append(test_articles[i])
            break

In [ ]:
print((matched_topic[0][0]))

In [74]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
   

    token = nltk.word_tokenize(doc)
    tag = nltk.pos_tag(token)
    nltk2wordnet = [(i[0], convert_tag(i[1])) for i in tag]
    output = [wn.synsets(i, z)[0] for i, z in nltk2wordnet if len(wn.synsets(i, z))>0]

    return output


def similarity_score(s1, s2):
   
    # Your Code Here
    list1 = []
    # For each synset in s1
    for a in s1:
        # finds the synset in s2 with the largest similarity value
        list1.append(max([i.path_similarity(a) for i in s2 if i.path_similarity(a) is not None]))

    output = sum(list1)/len(list1)

    return output


def document_path_similarity(doc1, doc2):
    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)
    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2






In [ ]:
t = np.zeros((len(matched_topic[0]),len(matched_topic[0])))
t             

In [ ]:
for i in range(len(matched_topic[0])):
              doc1 = test_articles[matched_topic[0][i]]
              for j in range(i, len(matched_topic[0])):
                  doc2 = test_articles[matched_topic[0][j]]
                  if i == j:
                      t[i][j]= -1
                      continue
                  t[i][j] = document_path_similarity(doc1, doc2)
                  t[j][i] = t[i][j]
                  print(t[i][j], end= '') 

In [74]:
 print(len(matched_topic[5]))

282


In [ ]:
t[0]

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = TfidfVectorizer(stop_words='english')
count_vectorizer = TfidfVectorizer()
sparse_matrix = count_vectorizer.fit_transform(matched_topic[5])

In [84]:
from sklearn.metrics.pairwise import cosine_similarity
a=[]
a= cosine_similarity( sparse_matrix, sparse_matrix)

In [85]:
for i in range(len(a)):
    for k in range(len(a[i])):
        if a[i][k] > 0.5 and i!=k:
            print("(",i,k,")", end = ' ')
            
            
            

( 9 38 ) ( 12 102 ) ( 31 32 ) ( 32 31 ) ( 32 34 ) ( 34 32 ) ( 34 132 ) ( 35 49 ) ( 37 121 ) ( 37 122 ) ( 38 9 ) ( 38 167 ) ( 38 210 ) ( 38 281 ) ( 40 128 ) ( 43 52 ) ( 43 126 ) ( 43 130 ) ( 48 127 ) ( 48 163 ) ( 49 35 ) ( 52 43 ) ( 52 126 ) ( 52 129 ) ( 52 130 ) ( 54 160 ) ( 76 78 ) ( 78 76 ) ( 85 231 ) ( 88 92 ) ( 92 88 ) ( 102 12 ) ( 115 197 ) ( 121 37 ) ( 121 122 ) ( 122 37 ) ( 122 121 ) ( 126 43 ) ( 126 52 ) ( 126 129 ) ( 126 130 ) ( 127 48 ) ( 128 40 ) ( 129 52 ) ( 129 126 ) ( 129 130 ) ( 130 43 ) ( 130 52 ) ( 130 126 ) ( 130 129 ) ( 132 34 ) ( 144 257 ) ( 147 161 ) ( 155 277 ) ( 160 54 ) ( 161 147 ) ( 163 48 ) ( 167 38 ) ( 167 181 ) ( 167 281 ) ( 181 167 ) ( 197 115 ) ( 206 272 ) ( 210 38 ) ( 231 85 ) ( 257 144 ) ( 261 262 ) ( 262 261 ) ( 272 206 ) ( 277 155 ) ( 281 38 ) ( 281 167 ) 

In [91]:
print(matched_topic[5][12])

in view of the covid19 pandemic the state government as an austerity measure on tuesday ordered deferment of salaries of ministers mlas and government officials in view of the adverse economic impact caused due to the lockdown and expenditures incurred to control the covid19 the government has opted for deferment of wages and salaries an official release said there would be a 70per cent deferment in gross salary of the chief minister ministers mlas chairperson of corporations and elected representatives of all local bodies similarly there would be a deferment of 50 per cent of gross salary of ias ips and ifs officers with immediate effect


In [90]:
print(matched_topic[5][102])

there shall be a deferment of 50 in gross salary in respect of the all india service officers — indian administrative service indian police service and indian forest service the odisha government on tuesday ordered austerity measures in view of the economic impact caused due to the covid19 lockdown and expenditure incurred to control the pandemic according to the government’s directive there shall be a deferment of 70 in the gross salary in respect of the chief minister ministers mlas chairperson of all corporations and elected representatives of all urban local bodies ulbs there shall be a deferment of 50 in gross salary in respect of the all india service officers — indian administrative service ias indian police service ips and indian forest service ifs this order will come into force with immediate effect according to an official release
